# Programming Assignment - 3
---

## Name:


In [248]:
#Import required packages here
import numpy as np

## Question 1

Use the Gaussian elimination with scaled row-partial pivoting code to answer the following.

In [249]:
## Gaussian Elimination: Scaled Row Pivoting
## This function is based on the pseudo-code on page-148 in the Text by Kincaid and Cheney
def GE_srpp(X, verbose=False):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    A = np.copy(X)
    m,n = A.shape
    swap=0;
    
    # The initial ordering of rows
    p = list(range(m))
    if verbose:
        print("permutation vector initialized to: ",p)
    
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A), axis=1) 
    
    # Start the k-1 passes of Guassian Elimination on A
    for k in range(m-1):              
        if verbose:
            print("Scaling Vector: ",s)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange elements in the permutation vector if needed
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
            swap+=1;
                        
        if verbose:
            print("\nPivot Element: {0:.4f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is singular or there is a zero pivot.")        
        # Check the new order of rows
        if verbose:
            print("permutation vector: ",p)
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, m):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]
            
            #Save the multiplier z in A itself. You can save this in L also
            A[p[i],k] = z
            
            #Elimination operation: Changes all elements in a row simultaneously
            A[p[i],k+1:] = A[p[i],k+1:] - z*A[p[k],k+1:]
            
        if verbose:
            print("\n After PASS {}=========: \n".format(k+1), A)
    return A, p, swap

In [250]:
## Example on page number 146 (Kincaid Cheney).
## Example solved manually in class
A = np.array([[2, 3, -6], [1,-6,8], [3, -2, 1]], dtype=float)
print("\n Given A: \n ",A)
A,p,swap =GE_srpp(A)
print("\n After Gaussian Elimination with RSPP: \n", A)
print("\n The permutation Vector is: \n", p)


 Given A: 
  [[ 2.  3. -6.]
 [ 1. -6.  8.]
 [ 3. -2.  1.]]

 After Gaussian Elimination with RSPP: 
 [[ 0.66666667  4.33333333 -6.66666667]
 [ 0.33333333 -1.23076923 -0.53846154]
 [ 3.         -2.          1.        ]]

 The permutation Vector is: 
 [2, 0, 1]


>- (A) Modify this code to write a function that solves a linear system Ax =b.  Test this in the case when   $𝑏=[3,1,1]^T$, and the matrix $A = [1~ 6 ~0; 2~ 1~ 0; 0 ~2~ 1]$. Only display the solution in the output.

In [251]:
def GE_srpp(X, verbose=False):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    A = np.copy(X)
    m,n = A.shape
    swap=0;
    
    # The initial ordering of rows
    p = list(range(m))
    if verbose:
        print("permutation vector initialized to: ",p)
    
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A), axis=1) 
    
    # Start the k-1 passes of Guassian Elimination on A
    for k in range(m-1):              
        if verbose:
            print("Scaling Vector: ",s)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange elements in the permutation vector if needed
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
            swap+=1;
                        
        if verbose:
            print("\nPivot Element: {0:.4f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is singular or there is a zero pivot.")        
        # Check the new order of rows
        if verbose:
            print("permutation vector: ",p)
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, m):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]
            
            #Save the multiplier z in A itself. You can save this in L also
            A[p[i],k] = z
            
            #Elimination operation: Changes all elements in a row simultaneously
            A[p[i],k+1:] = A[p[i],k+1:] - z*A[p[k],k+1:]
            
        if verbose:
            print("\n After PASS {}=========: \n".format(k+1), A)
        
        m=A.shape[0]
        U = np.triu(A[p,:])
    return A, p, swap, U


X = np.array([[1, 6, 0],
              [2, 1, 0], 
              [0, 2, 1]], dtype = float)
b = np.array([3, 1, 1], dtype = float).reshape(3, -1)
AugA = np.hstack((X, b))

def back_sub(U, b):
    #Check for A being upper triangular
    ## Or maybe not
    n = U.shape[0]
    #Solution will be saved in variable x
    x = np.zeros_like(b, dtype=float)

    #Back-substitution
    ##last variable is found first
    x[n-1] = b[n-1] / U[n-1,n-1]
    ## Find the remaining n-1 variables from last to first
    for k in range(n-2,-1,-1):
        known_sums = np.dot(U[k,k+1:],x[k+1:])
        x[k] = (b[k] - known_sums) / U[k,k]
     
    return x

A, p, swap, U = GE_srpp(AugA, verbose = False)
bnew = U[:, 3]
Unew = U[:, [0, 1, 2]]
back_sub(Unew, bnew).reshape(3, -1)

array([[0.27272727],
       [0.45454545],
       [0.09090909]])

>- (B) Modify this code to find the determinant of any square matrix A. Note that 
$$PA = LU \Rightarrow \text{det} A  = \pm \text{det} U .$$
The sign depends of the number of row-swaps in the elimination process.  Use this code to find the determinant of any $10 \times 10$ matrix that you randomly generate. Compare your result with the built-in NumPy method.

In [257]:
#Your code comes here

def det(A):
    m,n = A.shape
    newA, p, swap, U = GE_srpp(A)
    U = np.triu(newA[p,:])
    det = np.prod(np.diag(U))
    if swap % 2 == 0:
        det = det
    else:
        det = -1 * det
    return det
        
B = np.random.randint(1, 9, (10, 10))

print(det(B))

360000


>- (C) Modify the system-solver that you have created to find the inverse of a square matrix. Use this code to display the inverse of  A = [1 6 0; 2 1 0; 0 2 1].

In [253]:
# Your code comes here
X = np.array([[1, 6, 0],
              [2, 1, 0], 
              [0, 2, 1]], dtype = float)

AugX = np.hstack((X, np.eye(3)))

A, p, swap, U = GE_srpp(AugX)

bnew = U[:, [3, 4, 5]]

Unew1 = U[:, [0, 1, 2]]
bnew[:, [0]]

for i in range(3):
    x.append(back_sub(Unew1, bnew[:, [i]]))

invX = np.hstack((x[0], x[1], x[2]))
invX

array([[-0.09090909,  0.54545455,  0.        ],
       [ 0.18181818, -0.09090909,  0.        ],
       [-0.36363636,  0.18181818,  1.        ]])